In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
import pandas as pd
from pandas import HDFStore

from scipy import signal

In [18]:
# store = HDFStore("storeTraffic.h5")
# store

<class 'pandas.io.pytables.HDFStore'>
File path: storeTraffic.h5
/auto                      series       (shape->[1])  
/connTest                  series       (shape->[142])
/connTotal                 series       (shape->[852])
/connTrain                 series       (shape->[710])
/raw_conn_test             series       (shape->[142])
/raw_conn_train            series       (shape->[710])

In [2]:
raw_data = pd.read_csv("10min_workload.csv")
n_row = raw_data.size
workload = np.asarray([raw_data.ix[i][0]  for i in range(1,n_row)])

In [23]:
workload = np.array(store["raw_conn_train"])
print(workload.shape)

(710L,)


In [9]:
#tinh autocorrelation
# dich ve goc toa do truoc khi chay
n = len(workload)
variance = workload.var()
workload2= workload-workload.mean()
r=np.correlate(workload2, workload2,mode = 'full')[-n:]
# assert np.allclose(r, np.array([(workload[:n-k]*workload[-(n-k):]).sum() for k in range(n)]))
result = r/(n*variance)
# result = r/(variance*(np.arange(n, 0, -1)))
plt.plot(np.array(range(0,n))/144.0,result)

plt.figure(2)
plt.plot(np.array(range(0,workload2.size))/144.0, workload);
plt.show()

In [21]:
# tinh chu ky dua vao autcorrelation
[pksh,lcsh] = findpeaks(autocor);
short = mean(diff(lcsh))/fs

[pklg,lclg] = findpeaks(autocor, ...
    'MinPeakDistance',ceil(short)*fs/2,'MinPeakheight',0.1);
long = mean(diff(lclg))/fs

hold on
pks = plot(lags(lcsh)/fs,pksh,'or', ...
    lags(lclg)/fs,pklg+0.05,'vk');
hold off
legend(pks,[repmat('Period: ',[2 1]) num2str([short;long],0)]) 

SyntaxError: invalid syntax (<ipython-input-21-6ef0d6436813>, line 5)

In [10]:
#circular autocorrelation

lag = range(1,144*60)
autocorr = [np.correlate(workload,np.roll(workload,-i))[0] / workload.size for i in lag]
plt.plot(np.array(range(1,144*60))/144.0,autocorr)
# plt.figure(2)
# dif = np.diff(autocorr,n=2)
# plt.plot(dif)

plt.show()

In [7]:
# mien tan so
workload2= workload-workload.mean()
fs = 6*24
f, Pxx_den = signal.periodogram(workload, fs)
plt.figure(4)
plt.plot(f, Pxx_den)
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD [V**2/Hz]')
plt.show()



In [26]:
for i in workload:
    print i , ", " ,

IndentationError: expected an indented block (<ipython-input-26-ff63fef5ac8c>, line 2)

In [24]:
print dif[110:120]


[  1.36154059e-03   8.25641393e-04  -1.22591144e-03   3.15116633e-04
  -9.16465394e-05   4.20553850e-04   1.07478507e-03  -4.59463972e-03
   3.38214004e-03  -2.46598093e-04]


In [25]:
print dif[199:205]

[ 0.00174943 -0.00361615  0.00044671  0.00044636  0.00050024  0.00041486]


In [14]:
np.array(range(1,5)).var()

1.25

In [38]:
np.roll([1,2,3],-1)

array([2, 3, 1])